<a href="https://colab.research.google.com/github/digvijaylp/submit/blob/main/SuBMIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SuBMIT: Structure Based Model(s) Input Toolkit
## Package to generate Coarse-Grained Structure (.gro/.pdb) and Topology (.top/.xml) for using Augmented Structure Based Models MD Simulations on GROMACS and OpenSMOG (OpenMM based)

 ![alt text](https://github.com/sglabncbs/submit/blob/main/examples/SuBMIT.png)

In [1]:
import ipywidgets as widgets
from IPython.display import display


### Select preset models:

In [2]:
models={ "custom":"SuBMIT custom SBM Options",\
            "clementi2000":"Clementi et. al. 2000 CA-only model. 10.1006/jmbi.2000.3693",\
	        "afsar2008":"Zarrine-Afsar et. al. 2008 CA-only + hydrophobic model with . 10.1073/pnas.0801874105",\
	        "azia2009":"Azia 2009 CB-CA + Debye-Huckel model. 10.1016/j.jmb.2009.08.010",\
	        "pal2019":"Pal & Levy 2019 Protein CB-CA & RNA/DNA P-S-B model. 10.1371/journal.pcbi.1006768",\
	        "reddy2017":"Reddy & Thirumalai 2017 SOP-SC CA-CB. 10.1021/acs.jpcb.6b13100",\
	        "denesyuk2013":"Denesyuk & Thirumalai 2013 Three Interaction Site TIS P-S-B model. 10.1021/jp401087x",\
	        "chakraborty2018":"Chakraborty & Thirumalai 2018 Three Interaction Site TIS P-S-B model. 10.1021/acs.jctc.8b00091",\
	        "baul2019":"Baul et. al. 2019 SOP-SC-IDP CA-CB. 10.1021/acs.jpcb.9b02575",\
	        "baidya2022":"Baidya & Reddy 2022 SOP-SC-IDP CA-CB. 10.1021/acs.jpclett.2c01972",\
        	"baratam2024":"Baratam & Srivastava 2024 SOP-MULTI CA-CB. 10.1101/2024.04.29.591764",\
        	"sop_idr":"Reddy-Thiruamalai(SOPSC) + Baidya-Reddy(SOPIDP) hybrid CA-CB",\
        	"girirao2016":"Generate multi-basin .top/.xml file",\
        	"banerjee2023":"Banerjee & Gosavi 2023 Self-Peptide model. 10.1021/acs.jpcb.2c05917",\
        	"virusassembly":"Preset for structure based virus assembly (dlprakash 2025)",\
}
selectmodel = widgets.Dropdown(description='Models',options=list(models.keys()),value="custom",disabled=False)
button = widgets.Button(description='Model information',button_style='info',tooltip='model info',icon='info')
def description(btn):print (models[selectmodel.value],flush=True)
button.on_click(description)
display (widgets.HBox([selectmodel,button]))


SuBMIT custom SBM Options


### Coarse-graining parameters:-

In [16]:
if "custom" in selectmodel.value:
    disable_option=False
    prot_cg = widgets.RadioButtons(options=["1-bead/res CA-only", "2-bead/res CB+CA"],value="2-bead/res CB+CA",description='',disabled=disable_option,)
    nucl_cg = widgets.RadioButtons(options=["1-bead/res P-only", "3-bead/res P-S-B"],value="3-bead/res P-S-B",description='',disabled=disable_option,)
    display (widgets.VBox([widgets.Label('1. Amino-acid CG-level:'),prot_cg]))
    display (widgets.VBox([widgets.Label('2. Nucleotide CG-level'),nucl_cg]))
    #display (widgets.VBox([widgets.Label("1b. Position of C-alpha bead:"),CA_position]))
    #display (widgets.VBox([widgets.Label("1c. Position of C-beta bead:"),CB_position]))
else:
    print ("NOTE: CG paramters are not supported with preset models")
    disable_option=True


In [42]:
if "custom" in selectmodel.value:
    print ("\n1. Amino-acid bead positions.")
    CA_position = widgets.Dropdown(options=["CA atom", "Amino-acid backbone COM"],value="CA atom",disabled=disable_option,description='')
    if prot_cg.value.startswith('2'):
      CB_position = widgets.Dropdown(options=["CB atom", "Amino-acid sidechain COM","Farthest sidechain non-H atom"],value="CB atom",disabled=disable_option,description='')
      display (widgets.HBox([\
                widgets.VBox([widgets.Label("1a. Position of C-alpha bead:"),CA_position]),\
                widgets.VBox([widgets.Label("1b. Position of C-beta bead:"),CB_position])\
             ]))
    else: display (widgets.VBox([widgets.Label("1a. Position of C-alpha bead:"),CA_position]))
    print ("\n2. Nucleotide bead positions.")
    P_position = widgets.Dropdown(description='',options=['P-atom','OP1 atom', 'OP2 atom'],value="P-atom",disabled=False)
    if nucl_cg.value.startswith('3'):
      S_position = widgets.Dropdown(description='',options=["C1'-atom","C2'-atom","C3'-atom","C4'-atom","O4'-atom","C5'-atom"],value="C4'-atom",disabled=False)
      display (widgets.HBox([\
            widgets.VBox([widgets.Label("2a. Position of P bead:"),P_position]),\
            widgets.VBox([widgets.Label("2b. Position of S bead:"),S_position])\
      ]))
      Bpu_position = widgets.Dropdown(description='',options=["N1","C2","N3","C4","C5","C6","N7","C8","N9","COM"],value="COM",disabled=False)
      Bpy_position = widgets.Dropdown(description='',options=["N1","C2","N3","C4","C5","C6","COM"],value="COM",disabled=False)
      display (widgets.HBox([\
                             widgets.VBox([widgets.Label("2c. Position of A/G B bead:"),Bpu_position]),\
                             widgets.VBox([widgets.Label("2d. Position of C/U/T B bead:"),Bpy_position])\
                             ]))
    else: display (widgets.VBox([widgets.Label("2a. Position of P bead:"),P_position]))


else:
    print ("NOTE: CG paramters are not supported with preset models")
    disable_option=True



1. Amino-acid bead positions.



2. Nucleotide bead positions.
